In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

# import boost_histogram as bh
# import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf

Load background data from the run of interest

In [ ]:
bkg_dg = DataGroup('$CAGE_SW/processing/cage.json', load=True)
bkg_run = 'run==136 and skip==False'
# bkg_run = 'run==130 and skip==False'
# bkg_run = 'run==34 and skip==False'
bkg_dg.fileDB.query(bkg_run, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(bkg_dg.fileDB[view_cols])

In [ ]:
user = False
hit = True

Load alpha data from the run of interest

In [ ]:
alpha_dg = DataGroup('$CAGE_SW/processing/cage.json', load=True)
alpha_run = 'run==143 and skip==False'
# alpha_run = 'run==143 and skip==False'
# alpha_run = "run>50 and runtype=='alp'"
# alpha_run = 'run==50 and skip==False'
alpha_dg.fileDB.query(alpha_run, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(alpha_dg.fileDB[view_cols])

Plot raw energy histogram of the background data, then alpha data, from DSP files
First get dsp data, runtimes so we can normalize by time

First get file list and load background energy data (numpy array)

In [ ]:
# get file list and load background energy data

lh5_dir = bkg_dg.lh5_user_dir if user else bkg_dg.lh5_dir

# if hit files
if hit == True:
    bkg_file_list = lh5_dir + bkg_dg.fileDB['hit_path'] + '/' + bkg_dg.fileDB['hit_file']
    bkg_df = lh5.load_dfs(bkg_file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
else:
    bkg_file_list = lh5_dir + bkg_dg.fileDB['dsp_path'] + '/' + bkg_dg.fileDB['dsp_file']
    bkg_df = lh5.load_dfs(bkg_file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')
    
print(bkg_file_list)

bkg_rt_min = bkg_dg.fileDB['runtime'].sum()
bkg_u_start = bkg_dg.fileDB.iloc[0]['startTime']
bkg_t_start = pd.to_datetime(bkg_u_start, unit='s') # str
print(f'Background Runtime: {bkg_rt_min} minutes')

Get file list for alpha data

In [ ]:
# get file list and load alpha data 

lh5_dir = alpha_dg.lh5_user_dir if user else alpha_dg.lh5_dir

# if hit files
if hit == True:
    alpha_file_list = lh5_dir + alpha_dg.fileDB['hit_path'] + '/' + alpha_dg.fileDB['hit_file']
    alpha_df = lh5.load_dfs(alpha_file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
else:
    alpha_file_list = lh5_dir + alpha_dg.fileDB['dsp_path'] + '/' + alpha_dg.fileDB['dsp_file']
    alpha_df = lh5.load_dfs(alpha_file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')
    
print(alpha_file_list)

alpha_rt_min = alpha_dg.fileDB['runtime'].sum()
alpha_u_start = alpha_dg.fileDB.iloc[0]['startTime']
alpha_t_start = pd.to_datetime(alpha_u_start, unit='s') # str
print(f'Alpha Runtime: {alpha_rt_min} minutes')

Make a baseline cut

In [ ]:
blo, bhi, bpb = 9500, 10000, 1
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(bkg_df['bl'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

bl_cut_lo, bl_cut_hi = 9700,9760

plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
blo, bhi, bpb = 9500, 10000, 1
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(alpha_df['bl'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

bl_cut_lo, bl_cut_hi = 9700,9760

plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
# BL Cut

bkg_df_cut = bkg_df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()
alpha_df_cut = alpha_df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()

Create background histogram

In [ ]:
# Create background histogram, normalized by runtime
# elo, ehi, epb = 0, 25000, 10 
elo, ehi, epb = 0, 6000, 2
etype = 'trapEftp_cal'

bkg_ene_cal = bkg_df_cut[etype]
bkg_hist, bins, _ = pgh.get_hist(bkg_ene_cal, range=(elo, ehi), dx=epb)

# normalize by runtime in seconds
bkg_hist_rt = np.divide(bkg_hist, (bkg_rt_min))

Create alpha histogram

In [ ]:
# Create alpha histogram, normalized by runtime
etype = 'trapEftp_cal'
# elo, ehi, epb = 0, 25000, 10
elo, ehi, epb = 0, 6000, 2 

alpha_ene_cal = alpha_df_cut[etype]
alpha_hist, bins, _ = pgh.get_hist(alpha_ene_cal, range=(elo, ehi), dx=epb)

# normalize by runtime in seconds
alpha_hist_rt = np.divide(alpha_hist, (alpha_rt_min))

Plot background spectrum

In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
# Plot the histograms
plt.semilogy(bins[1:], bkg_hist_rt, ds='steps', c='b', lw=1,
            label=f'{etype}, {bkg_rt_min:.2f} mins')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / min', ha='right', y=1)
plt.ylim(0.003,50)
plt.legend()
plt.title(f'Background run: {bkg_run}')
plt.show()

Plot alpha spectrum

In [ ]:
# Plot the alpha histogram
plt.semilogy(bins[1:], alpha_hist_rt, ds='steps', c='b', lw=1,
            label=f'{etype}, {alpha_rt_min:.2f} mins')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / min', ha='right', y=1)
plt.ylim(0.003,50)
plt.xlim(0, 100)
plt.legend()
plt.title(f'Alpha run: {alpha_run}')
plt.show()

Subtract the Background histogram from the alpha histogram

In [ ]:
hist_bkg_sub = alpha_hist_rt - bkg_hist_rt

Plot background-subtracted histogram 

In [ ]:
# Plot the background-subtracted histogram
plt.semilogy(bins[1:], hist_bkg_sub, ds='steps', c='b', lw=1,
            label=f'{etype}, background subtracted')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / min', ha='right', y=1)
plt.xlim(0, 100)
plt.legend()
plt.title(f'Background subtracted, Alpha run: {alpha_run}')
plt.show()

Use BL cut from above, create A/E

In [ ]:
# Background A_10/trapEmax vs trapEmax (A/E vs E)


# add new A/E column
# bkg_df_cut['AoE_cal'] = bkg_df_cut['A_10'] / bkg_df_cut['trapEftp_cal']
bkg_df_cut['AoE_cal'] = np.divide(bkg_df_cut['A_10'], bkg_df_cut['trapEftp_cal'])

# add new A/E column
alpha_df_cut['AoE_cal'] = alpha_df_cut['A_10'] / alpha_df_cut['trapEftp_cal']

In [ ]:
#A/E Histogram

alo, ahi, apb = 0.0, 0.15, 0.00015
# elo, ehi, epb = 0, 10000, 10
# elo, ehi, epb = 0, 3000, 10
elo, ehi, epb = 0, 6000, 5
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

bkg_aoe_hist, xedges, yedges = np.histogram2d(bkg_df_cut[etype], bkg_df_cut['AoE'], bins=[nbx, nby], range=([elo, ehi], [alo, ahi]))

X, Y = np.mgrid[elo:ehi:nbx*1j, alo:ahi:nby*1j]

bkg_aoe_hist_norm = np.divide(bkg_aoe_hist, (bkg_rt_min))

fig = plt.figure(figsize=(12, 8))

pcm1 = plt.pcolormesh(X, Y, bkg_aoe_hist_norm, norm=LogNorm(0.002, 0.2))

plt.xlabel(f'{etype} (keV)')
plt.title(f'Background A/E ({bkg_rt_min} min run)')

CB = plt.colorbar(pcm1)
CB.set_label('Counts/min')

plt.show()


In [ ]:
#A/E Histogram

alo, ahi, apb = 0.0, 0.15, 0.00015
# elo, ehi, epb = 0, 10000, 10
# elo, ehi, epb = 0, 3000, 10
elo, ehi, epb = 0, 6000, 5
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

alpha_aoe_hist, xedges, yedges = np.histogram2d(alpha_df_cut[etype], alpha_df_cut['AoE'], bins=[nbx, nby], range=([elo, ehi], [alo, ahi]))

X, Y = np.mgrid[elo:ehi:nbx*1j, alo:ahi:nby*1j]

alpha_aoe_hist_norm = np.divide(alpha_aoe_hist, (alpha_rt_min))

fig = plt.figure(figsize=(12, 8))

pcm = plt.pcolormesh(X, Y, alpha_aoe_hist_norm, norm=LogNorm(0.002, 0.2))

plt.xlabel(f'{etype} (keV)')
plt.title(f'Alpha A/E ({alpha_rt_min:.2f} min run)')

CB = plt.colorbar(pcm)
CB.set_label('Counts/min')


plt.show()

In [ ]:
aoe_backgroundSub = bkg_aoe_hist_norm - alpha_aoe_hist_norm
aoe_backgroundSub_1 = alpha_aoe_hist_norm - bkg_aoe_hist_norm
print(np.sum(aoe_backgroundSub))
print(np.sum(alpha_aoe_hist_norm))
print(np.sum(bkg_aoe_hist_norm))

In [ ]:
fig = plt.figure(figsize=(12, 8))

pcm2 = plt.pcolormesh(X, Y, aoe_backgroundSub.clip(0.), norm=LogNorm(0.002, 0.2))

plt.xlabel(f'{etype} (keV)')
plt.title(f'Background-subtracted (bkg-alp) A/E')


CB = plt.colorbar(pcm2)
CB.set_label('Counts/min')

plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 8))

pcm2 = plt.pcolormesh(X, Y, aoe_backgroundSub_1.clip(0.), norm=LogNorm(0.002, 0.2))

plt.xlabel(f'{etype} (keV)')
plt.title(f'Background-subtracted (alp-bkg) A/E')


CB = plt.colorbar(pcm2)
CB.set_label('Counts/min')

plt.show()

DCR

In [ ]:
# #creat new DCR
# const = -0.0015
# const2 = -0.0000015
# # const2 = -0.0000000000003
# bkg_df_cut['dcr_new'] = bkg_df_cut['dcr'] + const*(bkg_df_cut['trapEftp_cal']) + const2*(bkg_df_cut['trapEftp_cal'])**2

# alpha_df_cut['dcr_new'] = alpha_df_cut['dcr'] + const*(alpha_df_cut['trapEftp_cal']) + const2*(alpha_df_cut['trapEftp_cal'])**2

In [ ]:
#creat new DCR
const = -0.0003
const2 = -0.0000003
# const2 = -0.0000000000003
bkg_df_cut['dcr_new'] = bkg_df_cut['dcr'] + const*(bkg_df_cut['trapEftp']) + const2*(bkg_df_cut['trapEftp'])**2 

alpha_df_cut['dcr_new'] = alpha_df_cut['dcr'] + const*(alpha_df_cut['trapEftp']) + const2*(alpha_df_cut['trapEftp'])**2 

In [ ]:
#bkg DCR

dlo, dhi, dpb = -20., 40, 0.1
elo, ehi, epb = 0, 6000, 5
        

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

bkg_dcr_hist, xedges, yedges = np.histogram2d(bkg_df_cut[etype], bkg_df_cut['dcr_new'], bins=[nbx, nby], range=([elo, ehi], [dlo, dhi]))

X, Y = np.mgrid[elo:ehi:nbx*1j, dlo:dhi:nby*1j]

bkg_dcr_hist_norm = np.divide(bkg_dcr_hist, (bkg_rt_min))

fig = plt.figure(figsize=(12, 8))
ax = plt.subplot()

ax.text(0.95, 0.83, f'{np.sum(bkg_dcr_hist_norm):.1f} counts/min', verticalalignment='bottom',
                    horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})


pcm1 = plt.pcolormesh(X, Y, bkg_dcr_hist_norm, norm=LogNorm(0.002, 0.2))

plt.xlabel(f'{etype} (keV)')
plt.title(f'Background DCR ({bkg_rt_min} min run)')

CB = plt.colorbar(pcm1)
CB.set_label('Counts/min')

plt.show()

In [ ]:
#Alpha DCR

dlo, dhi, dpb = -20., 40, 0.1
elo, ehi, epb = 0, 6000, 5
        

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist, xedges, yedges = np.histogram2d(alpha_df_cut[etype], alpha_df_cut['dcr_new'], bins=[nbx, nby], range=([elo, ehi], [dlo, dhi]))

X, Y = np.mgrid[elo:ehi:nbx*1j, dlo:dhi:nby*1j]

alpha_dcr_hist_norm = np.divide(alpha_dcr_hist, (alpha_rt_min))

fig = plt.figure(figsize=(12, 8))
ax = plt.subplot()

ax.text(0.95, 0.83, f'{np.sum(alpha_dcr_hist_norm):.1f} counts/min', verticalalignment='bottom',
                    horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

pcm1 = plt.pcolormesh(X, Y, alpha_dcr_hist_norm, norm=LogNorm(0.002, 0.2))


plt.xlabel(f'{etype} (keV)')
plt.title(f'Alpha DCR ({alpha_rt_min} min run)')

CB = plt.colorbar(pcm1)
CB.set_label('Counts/min')

plt.show()

Create BG subtracted DCR hists

In [ ]:
dcr_backgroundSub = (bkg_dcr_hist_norm - alpha_dcr_hist_norm)
dcr_backgroundSub_1 = (alpha_dcr_hist_norm - bkg_dcr_hist_norm)

clipped = dcr_backgroundSub_1.clip(0)
print(np.min(dcr_backgroundSub_1))
print(np.sum(alpha_dcr_hist_norm))
print(np.sum(alpha_dcr_hist_norm) - np.sum(bkg_dcr_hist_norm))
# print(np.sum(dcr_backgroundSub_1))

In [ ]:
print(np.sum(bkg_dcr_hist_norm) - np.sum(dcr_backgroundSub))
print(np.min(dcr_backgroundSub))

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot()


pcm2 = plt.pcolormesh(X, Y, dcr_backgroundSub, norm=LogNorm(0.002, 0.2))

ax.text(0.95, 0.83, f'{np.sum(dcr_backgroundSub):.1f} counts', verticalalignment='bottom',
                    horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.xlabel(f'{etype} (keV)')
plt.title(f'Background-subtracted (bkg-alp) DCR')


CB = plt.colorbar(pcm2)
CB.set_label('Counts/min')

plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.subplot()

pcm2 = plt.pcolormesh(X, Y, dcr_backgroundSub_1, norm=LogNorm(0.002, 0.2))

plt.xlabel(f'{etype} (keV)')
plt.title(f'Background-subtracted (alp-bkg) DCR')

# leg = plt.legend(f'{np.sum(dcr_backgroundSub_1)} counts', loc= 'upper right')
# leg.show()
ax.text(0.95, 0.83, f'{np.sum(dcr_backgroundSub_1):.1f} counts', verticalalignment='bottom',
                    horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

CB = plt.colorbar(pcm2)
CB.set_label('Counts/min')

plt.show()

In [ ]:
# with open('test3d_save_hist.pkl', 'wb') as f:
#     pl.dump(energy_aoe_dcr_hist, f)